In [55]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from torch.nn import ReLU, Linear, Dropout, CrossEntropyLoss, NLLLoss
import torch.nn as nn
from torch.optim import Adam
from torchsummary import summary
from torch import randn
import numpy as np

In [56]:
TrainBS = 128
TestBS = 64
Random_Seed = np.random.uniform()
torch.manual_seed(Random_Seed)

In [57]:
Train_Data = DataLoader(dataset = MNIST(train = True,
                                        root = '/data/',
                                        download = True,
                                        transform = ToTensor()),
                        batch_size = TrainBS,
                        shuffle = True)
Test_Data = DataLoader(dataset = MNIST(train = False,
                                        root = '/data/',
                                        download = True,
                                        transform = ToTensor()),
                        batch_size = TestBS,
                        shuffle = True)

In [58]:
class Teacher_Model(nn.Module):
    def __init__(self, num_classes = 10):
        super(Teacher_Model, self).__init__()
        self.fc1 = Linear(784, 784)         
        self.fc2 = Linear(784, 1200)
        self.fc3 = Linear(1200, num_classes)
        self.relu = ReLU()
        self.dropout = Dropout(p = 0.5)
    def forward(self,x):
        x = x.view (-1,784)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.fc3(x)

        return x

In [59]:
Model = Teacher_Model(num_classes = 10)
summary(Model, (1,1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 784]         615,440
           Dropout-2                  [-1, 784]               0
              ReLU-3                  [-1, 784]               0
            Linear-4                 [-1, 1200]         942,000
           Dropout-5                 [-1, 1200]               0
              ReLU-6                 [-1, 1200]               0
            Linear-7                   [-1, 10]          12,010
Total params: 1,569,450
Trainable params: 1,569,450
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 5.99
Estimated Total Size (MB): 6.04
----------------------------------------------------------------


In [60]:
Model = Teacher_Model()
optimizer = Adam(params = Model.parameters(),lr=1e-2)  
loss_function = CrossEntropyLoss()

In [62]:
epochs = 1 
for epoch in range(epochs):
    Model.train()
    for batch_idx, (data,target) in enumerate(Train_Data):
        optimizer.zero_grad()
        pred = Model(data)
        loss = loss_function (pred,target)
        loss.backward()
        optimizer.step()
        tmp,pred = pred.max(1)
        correct = 0
        correct += pred.eq(target).sum()
        print('train epoch:{} [{}/{} ({:.0f}%)]\t Loss:{:.6f}\t Acc:{:.6f}'.format(
                epoch,batch_idx*len(data),len(Train_Data.dataset),
                100.*batch_idx/len(Train_Data),loss.item(), 100.*correct/128))

train epoch:0 [0/60000 (0%)]	 Loss:0.558514	 Acc:86.718750
train epoch:0 [128/60000 (0%)]	 Loss:0.926196	 Acc:87.500000
train epoch:0 [256/60000 (0%)]	 Loss:0.739337	 Acc:89.062500
train epoch:0 [384/60000 (1%)]	 Loss:0.586004	 Acc:86.718750
train epoch:0 [512/60000 (1%)]	 Loss:0.715725	 Acc:85.156250
train epoch:0 [640/60000 (1%)]	 Loss:0.609133	 Acc:87.500000
train epoch:0 [768/60000 (1%)]	 Loss:0.512858	 Acc:84.375000
train epoch:0 [896/60000 (1%)]	 Loss:0.576244	 Acc:85.937500
train epoch:0 [1024/60000 (2%)]	 Loss:0.486361	 Acc:85.937500
train epoch:0 [1152/60000 (2%)]	 Loss:0.375317	 Acc:91.406250
train epoch:0 [1280/60000 (2%)]	 Loss:0.385579	 Acc:86.718750
train epoch:0 [1408/60000 (2%)]	 Loss:0.558676	 Acc:83.593750
train epoch:0 [1536/60000 (3%)]	 Loss:0.618531	 Acc:85.937500
train epoch:0 [1664/60000 (3%)]	 Loss:1.471763	 Acc:80.468750
train epoch:0 [1792/60000 (3%)]	 Loss:0.498670	 Acc:86.718750
train epoch:0 [1920/60000 (3%)]	 Loss:0.715192	 Acc:84.375000
train epoch:0 [204

train epoch:0 [16896/60000 (28%)]	 Loss:0.757797	 Acc:82.812500
train epoch:0 [17024/60000 (28%)]	 Loss:0.431130	 Acc:87.500000
train epoch:0 [17152/60000 (29%)]	 Loss:0.450655	 Acc:84.375000
train epoch:0 [17280/60000 (29%)]	 Loss:0.533318	 Acc:84.375000
train epoch:0 [17408/60000 (29%)]	 Loss:0.678473	 Acc:78.906250
train epoch:0 [17536/60000 (29%)]	 Loss:0.646684	 Acc:81.250000
train epoch:0 [17664/60000 (29%)]	 Loss:0.628997	 Acc:80.468750
train epoch:0 [17792/60000 (30%)]	 Loss:0.366479	 Acc:89.843750
train epoch:0 [17920/60000 (30%)]	 Loss:0.854530	 Acc:81.250000
train epoch:0 [18048/60000 (30%)]	 Loss:0.419209	 Acc:85.937500
train epoch:0 [18176/60000 (30%)]	 Loss:0.549184	 Acc:85.156250
train epoch:0 [18304/60000 (30%)]	 Loss:0.560333	 Acc:84.375000
train epoch:0 [18432/60000 (31%)]	 Loss:0.706439	 Acc:82.812500
train epoch:0 [18560/60000 (31%)]	 Loss:0.383380	 Acc:89.843750
train epoch:0 [18688/60000 (31%)]	 Loss:0.493503	 Acc:82.812500
train epoch:0 [18816/60000 (31%)]	 Loss:

train epoch:0 [33408/60000 (56%)]	 Loss:0.436679	 Acc:86.718750
train epoch:0 [33536/60000 (56%)]	 Loss:0.412039	 Acc:89.062500
train epoch:0 [33664/60000 (56%)]	 Loss:0.385726	 Acc:86.718750
train epoch:0 [33792/60000 (56%)]	 Loss:0.560313	 Acc:82.812500
train epoch:0 [33920/60000 (57%)]	 Loss:0.601274	 Acc:87.500000
train epoch:0 [34048/60000 (57%)]	 Loss:0.309827	 Acc:90.625000
train epoch:0 [34176/60000 (57%)]	 Loss:0.497384	 Acc:87.500000
train epoch:0 [34304/60000 (57%)]	 Loss:0.652565	 Acc:85.937500
train epoch:0 [34432/60000 (57%)]	 Loss:0.411625	 Acc:88.281250
train epoch:0 [34560/60000 (58%)]	 Loss:0.390185	 Acc:85.937500
train epoch:0 [34688/60000 (58%)]	 Loss:0.524117	 Acc:85.156250
train epoch:0 [34816/60000 (58%)]	 Loss:0.346658	 Acc:90.625000
train epoch:0 [34944/60000 (58%)]	 Loss:0.537789	 Acc:81.250000
train epoch:0 [35072/60000 (58%)]	 Loss:0.526284	 Acc:87.500000
train epoch:0 [35200/60000 (59%)]	 Loss:0.437650	 Acc:89.843750
train epoch:0 [35328/60000 (59%)]	 Loss:

train epoch:0 [50176/60000 (84%)]	 Loss:0.573834	 Acc:85.937500
train epoch:0 [50304/60000 (84%)]	 Loss:0.427122	 Acc:85.937500
train epoch:0 [50432/60000 (84%)]	 Loss:0.495878	 Acc:85.937500
train epoch:0 [50560/60000 (84%)]	 Loss:0.464094	 Acc:84.375000
train epoch:0 [50688/60000 (84%)]	 Loss:0.555813	 Acc:85.937500
train epoch:0 [50816/60000 (85%)]	 Loss:0.483222	 Acc:85.156250
train epoch:0 [50944/60000 (85%)]	 Loss:0.649617	 Acc:85.937500
train epoch:0 [51072/60000 (85%)]	 Loss:0.584635	 Acc:82.031250
train epoch:0 [51200/60000 (85%)]	 Loss:0.537020	 Acc:82.812500
train epoch:0 [51328/60000 (86%)]	 Loss:0.664822	 Acc:80.468750
train epoch:0 [51456/60000 (86%)]	 Loss:0.386562	 Acc:85.156250
train epoch:0 [51584/60000 (86%)]	 Loss:0.535955	 Acc:85.156250
train epoch:0 [51712/60000 (86%)]	 Loss:0.863201	 Acc:89.062500
train epoch:0 [51840/60000 (86%)]	 Loss:0.702095	 Acc:82.812500
train epoch:0 [51968/60000 (87%)]	 Loss:0.586888	 Acc:82.031250
train epoch:0 [52096/60000 (87%)]	 Loss:

In [65]:
class Student_Model(nn.Module):
    def __init__(self, num_class = 10):
        super(Student_Modle,self).__init__()
        self.fc1 = Linear(784, 20)
        self.fc2 = Lineat(20, num_class)
        self.relu = ReLU()
        self.dropout = Dropout(p = 0.5)
    def forward(self,x):
        x = x.view(-1,784)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        return x

In [66]:
Model = Student_Model()
optimizer = Adam(params = Model.parameters(),lr=1e-2)  
loss_function = CrossEntropyLoss()

NameError: name 'Student_Modle' is not defined

In [67]:
epochs = 1
for epoch in range(epochs):
    Model.train()
    for batch_idx, (data,target) in enumerate(Train_Data):
        optimizer.zero_grad()
        pred = Model(data)
        loss = loss_function (pred,target)
        loss.backward()
        optimizer.step()
        tmp,pred = pred.max(1)
        correct = 0
        correct += pred.eq(target).sum()
        print('train epoch:{} [{}/{} ({:.0f}%)]\t Loss:{:.6f}\t Acc:{:.6f}'.format(
                epoch,batch_idx*len(data),len(Train_Data.dataset),
                100.*batch_idx/len(Train_Data),loss.item(), 100.*correct/128))
        

train epoch:0 [0/60000 (0%)]	 Loss:0.466930	 Acc:85.937500
train epoch:0 [128/60000 (0%)]	 Loss:0.657023	 Acc:85.156250
train epoch:0 [256/60000 (0%)]	 Loss:0.656125	 Acc:83.593750
train epoch:0 [384/60000 (1%)]	 Loss:0.314715	 Acc:89.843750
train epoch:0 [512/60000 (1%)]	 Loss:0.503966	 Acc:85.156250
train epoch:0 [640/60000 (1%)]	 Loss:0.520953	 Acc:82.031250
train epoch:0 [768/60000 (1%)]	 Loss:0.344924	 Acc:92.187500
train epoch:0 [896/60000 (1%)]	 Loss:0.389019	 Acc:89.062500
train epoch:0 [1024/60000 (2%)]	 Loss:0.397458	 Acc:88.281250
train epoch:0 [1152/60000 (2%)]	 Loss:0.623684	 Acc:82.812500
train epoch:0 [1280/60000 (2%)]	 Loss:0.391032	 Acc:91.406250
train epoch:0 [1408/60000 (2%)]	 Loss:0.628214	 Acc:85.156250
train epoch:0 [1536/60000 (3%)]	 Loss:0.480354	 Acc:87.500000
train epoch:0 [1664/60000 (3%)]	 Loss:0.316966	 Acc:90.625000
train epoch:0 [1792/60000 (3%)]	 Loss:0.531705	 Acc:88.281250
train epoch:0 [1920/60000 (3%)]	 Loss:0.484346	 Acc:85.156250
train epoch:0 [204

train epoch:0 [16896/60000 (28%)]	 Loss:0.442331	 Acc:88.281250
train epoch:0 [17024/60000 (28%)]	 Loss:0.584996	 Acc:89.062500
train epoch:0 [17152/60000 (29%)]	 Loss:0.378662	 Acc:86.718750
train epoch:0 [17280/60000 (29%)]	 Loss:0.381775	 Acc:92.968750
train epoch:0 [17408/60000 (29%)]	 Loss:0.817598	 Acc:83.593750
train epoch:0 [17536/60000 (29%)]	 Loss:0.877831	 Acc:85.937500
train epoch:0 [17664/60000 (29%)]	 Loss:0.472484	 Acc:85.156250
train epoch:0 [17792/60000 (30%)]	 Loss:0.476494	 Acc:90.625000
train epoch:0 [17920/60000 (30%)]	 Loss:0.961609	 Acc:85.156250
train epoch:0 [18048/60000 (30%)]	 Loss:0.473252	 Acc:87.500000
train epoch:0 [18176/60000 (30%)]	 Loss:1.398794	 Acc:89.843750
train epoch:0 [18304/60000 (30%)]	 Loss:0.795953	 Acc:87.500000
train epoch:0 [18432/60000 (31%)]	 Loss:0.408220	 Acc:86.718750
train epoch:0 [18560/60000 (31%)]	 Loss:0.934772	 Acc:85.937500
train epoch:0 [18688/60000 (31%)]	 Loss:0.280915	 Acc:90.625000
train epoch:0 [18816/60000 (31%)]	 Loss:

train epoch:0 [33792/60000 (56%)]	 Loss:0.557768	 Acc:82.031250
train epoch:0 [33920/60000 (57%)]	 Loss:0.541071	 Acc:86.718750
train epoch:0 [34048/60000 (57%)]	 Loss:0.458858	 Acc:86.718750
train epoch:0 [34176/60000 (57%)]	 Loss:0.517760	 Acc:87.500000
train epoch:0 [34304/60000 (57%)]	 Loss:0.832431	 Acc:85.156250
train epoch:0 [34432/60000 (57%)]	 Loss:0.488002	 Acc:83.593750
train epoch:0 [34560/60000 (58%)]	 Loss:0.411309	 Acc:87.500000
train epoch:0 [34688/60000 (58%)]	 Loss:0.446107	 Acc:82.812500
train epoch:0 [34816/60000 (58%)]	 Loss:0.288401	 Acc:91.406250
train epoch:0 [34944/60000 (58%)]	 Loss:0.423876	 Acc:83.593750
train epoch:0 [35072/60000 (58%)]	 Loss:0.587816	 Acc:82.031250
train epoch:0 [35200/60000 (59%)]	 Loss:0.472713	 Acc:88.281250
train epoch:0 [35328/60000 (59%)]	 Loss:0.948635	 Acc:80.468750
train epoch:0 [35456/60000 (59%)]	 Loss:0.753891	 Acc:78.906250
train epoch:0 [35584/60000 (59%)]	 Loss:0.744306	 Acc:86.718750
train epoch:0 [35712/60000 (59%)]	 Loss:

train epoch:0 [50560/60000 (84%)]	 Loss:0.500434	 Acc:83.593750
train epoch:0 [50688/60000 (84%)]	 Loss:0.530151	 Acc:81.250000
train epoch:0 [50816/60000 (85%)]	 Loss:0.439535	 Acc:89.062500
train epoch:0 [50944/60000 (85%)]	 Loss:1.388795	 Acc:85.156250
train epoch:0 [51072/60000 (85%)]	 Loss:0.642699	 Acc:81.250000
train epoch:0 [51200/60000 (85%)]	 Loss:0.418771	 Acc:85.937500
train epoch:0 [51328/60000 (86%)]	 Loss:0.655332	 Acc:87.500000
train epoch:0 [51456/60000 (86%)]	 Loss:0.278230	 Acc:92.187500
train epoch:0 [51584/60000 (86%)]	 Loss:0.305108	 Acc:89.062500
train epoch:0 [51712/60000 (86%)]	 Loss:0.431800	 Acc:85.937500
train epoch:0 [51840/60000 (86%)]	 Loss:0.442849	 Acc:88.281250
train epoch:0 [51968/60000 (87%)]	 Loss:0.535033	 Acc:83.593750
train epoch:0 [52096/60000 (87%)]	 Loss:0.614192	 Acc:86.718750
train epoch:0 [52224/60000 (87%)]	 Loss:0.798975	 Acc:83.593750
train epoch:0 [52352/60000 (87%)]	 Loss:0.578034	 Acc:88.281250
train epoch:0 [52480/60000 (87%)]	 Loss: